# Load DNS and mDNS datasets

In [7]:
import os
import sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import src.temporal_loader_v2 as tl
from src.utils import to_homogeneous


In [8]:
cuda_device = 4

if torch.cuda.is_available():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(cuda_device)
    
torch.manual_seed(42)

## Load Graphs

In [9]:
kg_path = lambda graph_name: f'../data/{graph_name}'
dataset = tl.DNS(root=kg_path('DNS_2m'), start=0, end=6, test_list=[7], balance_gt=True, domain_file='domains2.csv')

Total labeled 897635
Labeled node count for 0, 6: 31778
After balancing labeled count: 31282
Labeled node count for 0, 7: 2610


#### DNS

In [10]:
data = to_homogeneous(dataset.train_data) # training data
# test_data = to_homogeneous(dataset.test_data[0])
data.x.size()

torch.Size([1173558, 12])

#### GNN

In [11]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import RGCNConv, Linear


class RGCN(torch.nn.Module):
    def __init__(self, num_features, dim=16, num_classes=2, num_layers=2):
        super(RGCN, self).__init__()

        self.conv1 = RGCNConv(num_features, dim, num_relations=3)
        self.gcs = nn.ModuleList()
        self.num_layers = num_layers
        for i in range(1, num_layers):
            conv = RGCNConv(dim, dim, num_relations=3) 
            self.gcs.append(conv)
        self.lin = Linear(dim, num_classes)

    def forward(self, x, edge_index, edge_type):
        x = F.relu(self.conv1(x, edge_index, edge_type))
        x = F.dropout(x, training=self.training)
        for i in range(1, self.num_layers):
            x = F.relu(self.gcs[i-1](x, edge_index, edge_type))
            x = F.dropout(x, training=self.training)
        return self.lin(x)

In [12]:
cuda_device = 1
torch.manual_seed(42)
from src.utils import score

def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_type)
    mask = data.train_mask
    loss = F.cross_entropy(out[mask], data.y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(model, data):
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_type).argmax(dim=-1)

    accs = []
    for split in ['train_mask', 'val_mask']:
        mask = data[split]
        acc = (pred[mask] == data.y[mask]).sum() / mask.sum()
        accs.append(float(acc))
    return accs

def experiment(model,start,end,test_list, model_type):
    kg_path = lambda graph_name: f'../data/{graph_name}'

    dataset = tl.DNS(root=kg_path('DNS_2m'), start=start, end=end, test_list=test_list, balance_gt=False, domain_file='domains2.csv')
    data = to_homogeneous(dataset.train_data) # training data

    if torch.cuda.is_available():
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        torch.cuda.set_device(cuda_device)

        data, model = data.to(device), model.to(device)

    with torch.no_grad():  # Initialize lazy modules.
        out = model(data.x, data.edge_index, data.edge_type)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

    for epoch in range(0, 201):
        loss = train(model, data, optimizer)
        train_acc, val_acc = test(model,data)
        if epoch % 20 == 0:
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
                f'Val: {val_acc:.4f}')
        
    model.eval()
    for index, test_data in enumerate(dataset.test_data):
        test_data = to_homogeneous(test_data)
        test_data = test_data.to(device)
        with torch.no_grad():
            pred = model(test_data.x, test_data.edge_index, test_data.edge_type).argmax(dim=-1)
        mask = test_data['val_mask']
        scores = score(pred[mask],test_data.y[mask])
        with open("../results_copy.csv", "a") as logger:
            logger.write("{},{},{},{},".format(model_type,start,end,index))
            logger.write(",".join(str(x) for x in scores.values()))
            logger.write('\n')


        for metric, val in scores.items():
            print(metric, ':{:.4f}'.format(val))
    
for model_type in ['rgcn']:
    for i in range(5):
        # model_type='gcn'  
        data.x.size(1) 
        model = RGCN(data.x.size(1), dim=64, num_classes=2,
                  num_layers=2)
        experiment(model,i,i+6,[i+7,i+8], model_type)

Total labeled 897635
Labeled node count for 0, 6: 31778
Labeled node count for 0, 7: 2610
Labeled node count for 0, 8: 2083
Epoch: 000, Loss: 0.7034, Train: 0.6604, Val: 0.6658
Epoch: 020, Loss: 0.4749, Train: 0.7666, Val: 0.7739
Epoch: 040, Loss: 0.4318, Train: 0.8123, Val: 0.8179
Epoch: 060, Loss: 0.4089, Train: 0.8176, Val: 0.8225
Epoch: 080, Loss: 0.3960, Train: 0.8206, Val: 0.8228
Epoch: 100, Loss: 0.3929, Train: 0.8208, Val: 0.8198
Epoch: 120, Loss: 0.3893, Train: 0.8226, Val: 0.8195
Epoch: 140, Loss: 0.3875, Train: 0.8224, Val: 0.8201
Epoch: 160, Loss: 0.3873, Train: 0.8230, Val: 0.8228
Epoch: 180, Loss: 0.3855, Train: 0.8225, Val: 0.8206
Epoch: 200, Loss: 0.3866, Train: 0.8239, Val: 0.8212
tn, fp, fn, tp 1318 287 127 878
acc :0.8414
f1 :0.8431
auc :0.8474
prec :0.7536
recall :0.8736
fpr :0.1788
mi_f1 :0.8414
ma_f1 :0.8367
tn, fp, fn, tp 1072 237 114 660
acc :0.8315
f1 :0.8335
auc :0.8358
prec :0.7358
recall :0.8527
fpr :0.1811
mi_f1 :0.8315
ma_f1 :0.8246
Total labeled 897635
La